## Spark Assignment-III

#### Problem:

The file named “Universal.txt” contains a list of around 600 words, one per line, in alphabetical order and in lower case to be used as an universal set of words. A list of 1000 words made up of random characters with each word size limited to length 10 is available in “Stream.txt”.

Compute the number of words in Stream.txt that belong to the universal set using the bloom filter. Use a bit array of size 4K and k hash functions of the form:

(a*(hash(word)) + b) mod 4096, for random integers a and b. 

Check the actual number using a set membership algorithm and provide the false positive percentage if any, with varying k.


In [1]:
## setup
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark import SparkContext, SparkConf

conf = pyspark.SparkConf().setAppName('Lab2').setMaster('local')
sc = pyspark.SparkContext(conf=conf)

In [2]:
## Loading universal data
univ = sc.textFile("data/universal.txt")
univ.take(5)

['a', 'aadarinchade', 'able', 'about', 'abundant']

In [3]:
## no. of words in universal data
univ.count()

585

### Generating k hash functions

In [37]:
## k hash function gernerator
import random

def gen_kHash(k):
    khash = dict()
    for i in range(k):
        a=random.randint(0, 5000)
        b=random.randint(0, 5000)
        
        khash[i]= (a,b)
    
    return khash
        

### Bit Array

In [5]:
## Bit Array
bitArray = [0]*4096

### Generating Bloom filter from universal set.

In [6]:
## bloo  filter   
def gen_bloomFilter(k, univ):    
    kHash = gen_kHash(k)
    index=set()
    for a,b in kHash.values():
        temp = univ.map(lambda word: (a*hash(word) + b)%4096 )
        index=index.union(set(temp.collect()))
        
    return index, kHash 


In [7]:
### using 5 hash functions
k=5
index, kHash = gen_bloomFilter(k, univ)

In [8]:
kHash

{0: (1, 1), 1: (2, 2), 2: (0, 1), 3: (2, 2), 4: (1, 1)}

In [9]:
### No. of 1s in BitArray
for i in index:
    bitArray[i]=1

print("No. of 1's in bitArray:",sum(bitArray))

No. of 1's in bitArray: 987


### Filter Streams

In [10]:
stream = sc.textFile("data/Stream.txt")
stream.take(5)

['wgrr', 'atma', 'e', 'tnjf', 'i']

In [11]:
stream.count()

1003

In [13]:
## Filtering the stream using the bloom filter
filteredStream = stream.filter(lambda x: all( [ bitArray[(a*hash(x) + b)%4096]==1 for a,b in kHash.values() ] ) )

In [18]:
## No. of words predicted positive
pred_posi = filteredStream.count()
pred_posi

157

- Thus, 158 words in the stream belongs to universal dataset according our bloom filter.

#### Actual count of universal words in Streams

In [16]:
no_posi = univ.intersection(stream).count()
no_posi

8

In [17]:
n = stream.count()

In [21]:
print("Negatives:",n-no_posi)
print("False Positives:", pred_posi-no_posi)
print("False Positive percentage:",(pred_posi-no_posi)/n*100)

Negatives: 995
False Positives: 149
False Positive percentage: 14.855433698903289


### False positives with varying k

In [25]:
for k in range(1,20):
    bitArray = [0]*4096
    index, kHash = gen_bloomFilter(k, univ)
    for i in index:
        bitArray[i]=1
    filteredStream = stream.filter(lambda x: all( [ bitArray[(a*hash(x) + b)%4096]==1 for a,b in kHash.values() ] ) )
    pred_posi = filteredStream.count()
    print("For k=",k)
    print("Negatives:",n-no_posi)
    print("False Positives:", pred_posi-no_posi)
    print("False Positive percentage:",(pred_posi-no_posi)/n*100)
    print("------------------------------------------------------")

For k= 1
Negatives: 995
False Positives: 128
False Positive percentage: 12.761714855433699
------------------------------------------------------
For k= 2
Negatives: 995
False Positives: 152
False Positive percentage: 15.154536390827516
------------------------------------------------------
For k= 3
Negatives: 995
False Positives: 995
False Positive percentage: 99.20239282153538
------------------------------------------------------
For k= 4
Negatives: 995
False Positives: 146
False Positive percentage: 14.556331006979061
------------------------------------------------------
For k= 5
Negatives: 995
False Positives: 158
False Positive percentage: 15.752741774675972
------------------------------------------------------
For k= 6
Negatives: 995
False Positives: 159
False Positive percentage: 15.852442671984049
------------------------------------------------------
For k= 7
Negatives: 995
False Positives: 128
False Positive percentage: 12.761714855433699
----------------------------------

**Best No. of hash function was found to be k=1**  
**Least False Positive pecentage = 12%** 